In [ ]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-06-25 08:41:25--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-06-25 08:41:26--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               setup Colab for PySpark 3.0.3 and Spark NLP 3.1.1
-                   100%[==

In [ ]:
import pandas as pd
import numpy as np
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
import json
import pickle

In [ ]:
spark = sparknlp.start()
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

with open('/content/intents.json', 'rb') as f:
    intents_file = f.read()

intents_file = intents_file.split("\r\n")
intents = json.loads(intents_file)

TypeError: ignored

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer()\
    .setInputCols(["document"]) \
    .setOutputCol("token")

lemmatizer = LemmatizerModel.pretrained("lemma", "ta") \
        .setInputCols(["token"]) \
        .setOutputCol("lemma")

pipeline = Pipeline(stages=[document_assembler, tokenizer, lemmatizer])

example = spark.createDataFrame([['கட்சி வெற்றி பெற்றதோடு பெற்றத் ஓடு ஆட்சியில் உள்ள கட்சிக்கு ஒரு மாற்றுக் கட்சியாக வளர்ந்துள்ளது வளர்ந்த் உள்ளது .']], ["text"])

results = pipeline.fit(example).transform(example)

lemma download started this may take some time.
Approximate size to download 47.9 KB
[OK!]


In [ ]:
# lemmatization
results.select(F.explode(F.arrays_zip('lemma.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("lemma")).show(truncate=False)

+-------------+
|lemma        |
+-------------+
|கட்சி        |
|வெற்றி       |
|பெற்றதோடு    |
|பெற்றத்      |
|ஓடு          |
|ஆட்சி        |
|உள்          |
|கட்சிக்கு    |
|ஒரு          |
|மாற்றுக்     |
|கட்சியாக     |
|வளர்ந்துள்ளது|
|வளர்ந்த்     |
|உள்          |
|.            |
+-------------+

